# Sensitivity of rankings to criteria weights calculations

## Initial imports

In [13]:
import numpy as np
import pandas as pd
from copy import deepcopy
from scipy.stats import rankdata, stats
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from enum import Enum
import qgrid
from IPython.display import HTML
from wpt.math import *

## Load data

In [14]:
df = pd.read_excel("initial-data.xlsx", sheet_name=1, index_col=0)
weights = pd.read_excel("initial-data.xlsx").iloc[0,2:]
orig_weights = deepcopy(weights)
orig_ranks = rankdata(-df["Value"])
del df["Value"]
df.index.name=None
criteria = weights.index.tolist()
alternatives = df.index.tolist()
display(weights)
display(df)

Category A     0.48775
Category B    0.115658
Category C    0.045709
Threat        0.071927
Population    0.278955
Name: 0, dtype: object

,Category A,Category B,Category C,Threat,Population
Asset 23,0.667,0.0,0.000,0.767677,1.000000
Asset 19,0.667,0.0,0.000,0.696970,0.971020
Asset 15,0.667,0.4,0.000,0.070707,0.632661
Asset 25,0.500,0.0,0.000,0.080808,0.994852
Asset 26,0.667,0.4,0.000,0.707071,0.320068
Asset 14,0.667,0.4,0.000,0.545455,0.267750
Asset 22,0.667,0.0,0.000,0.010101,0.421111
Asset 21,0.500,0.0,0.000,0.393939,0.535068
Asset 12,0.667,0.4,0.000,0.313131,0.093636
Asset 3,0.667,0.0,0.000,0.939394,0.067646


# Main analysis

## Actual calculations

The basic idea here is:
* Loop over the number of alternatives you want rank change
    * Loop then over the amount of rank change each alternative would need to be counted as a "rank changing alternative"
        * Inside both of those loops, loop over each criteria, change it's weights up until you hit the required amount of rank changing alternatives
    * Then pick the criterion that required the least change to induce the given rank changes and report that as the minimum percent change required.

In [24]:
# Actual calculation!
## These are our loop variables
# This is the number of alts we want to look to change
nalts_list = [1,2,3,4,5,6,7]
# How many ranks should the alts change by in order to be considered important
nrk_changes_list = [1,2,3,4,5]
## End Loop variables

## Now we have dataframes that are going to report back, with the same row/column names
df_columns = ["{:d} Alts".format(i) for i in nalts_list]
df_index = ["{:d} Ranks".format(i) for i in nrk_changes_list]
## Done with dataframe indices

# This is the actual dataframe of how much criteria weights had to change by
# to induce the given number of alternatives to change by the given number of ranks
rval_df = pd.DataFrame(columns=df_columns,
                      index=df_index)
# The weights needed to induce the change
weights_df = pd.DataFrame(columns=df_columns, index=df_index, dtype=object)
# The resulting ranks that are caused by the change
new_ranks_df = pd.DataFrame(columns=df_columns, index=df_index, dtype=object)
# A is our matrix of ratings scores
A = df.values
# w0 is the original weights
w0 = orig_weights
# Loop over the number of alternatives we want to have a significant change
for count_dist in nalts_list:
    # Loop over the amount of rank change each alt must have, in order to be considered significant
    for min_dist in nrk_changes_list:
        # The amount of weight change for weight we need
        infos = []
        # The actual criteria weights we needed to use to get the prescribed rank changes. One for each criteria
        aweights = []
        # The scores of the alternatives that results from those new weights.  One for each critieria
        ascores = []
        # The resulting ranks from those scores
        aranks = []
        for i in range(len(w0)):
            # This loop is over each criteria.  For each critieria we adjust the weight of that
            # criteria up, and the remainder down until we first get the prescribed number of alternatives
            # that have the needed amount of rank change
            
            # This is the function that tweaks the weights, by upping one criterion's weight
            # and down weighting the others, while keeping proportionality
            f = lambda t : family_wts(t, w0, i)
            # The function to described the difference in rankings
            d = lambda r1, r2 : count_diffs_dist(r1, r2, min_dist=min_dist)
            # Calculate the actual needed change to induce the needed rank changes
            rval_info2 = firstTime(A, w0, f, d, count_dist,
                                  wt_combine_loc=i,
                                  wt_changes_percent=True,
                                  wt_changes_totaler=np.mean,
                                 return_param_only=False)
            # If successful, we add to our list of possible ways to get the desired rank change
            if rval_info2 is not None:
                rval = rval_info2['weight_changes_local']
                infos.append(rval)
                aweights.append(rval_info2["weights"])
                ascores.append(rval_info2["scores"])
                aranks.append(rval_info2["ranks"])
        if len(infos) > 0:
            # If we found at least one way to get the desired rank changes, find the smallest change
            # in criteria weights needed to induce the given rank changes
            # And record the results in rval_df, weights_df and new_ranks_df
            min_index = np.argmin(infos)
            quickest = infos[min_index]
            weights_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = aweights[min_index]
            new_ranks_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = aranks[min_index]
        else:
            quickest = None
        
        rval_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = quickest
        print("Smallest change to get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" requires weights change {:0.2%}".format(rval))
    print()
    print()
    

Smallest change to get at least 1 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 1 projects to change rank at least 2 requires weights change 3.36%
Smallest change to get at least 1 projects to change rank at least 3 requires weights change 16.54%
Smallest change to get at least 1 projects to change rank at least 4 requires weights change 18.09%
Smallest change to get at least 1 projects to change rank at least 5 requires weights change 19.90%


Smallest change to get at least 2 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 2 projects to change rank at least 2 requires weights change 3.88%
Smallest change to get at least 2 projects to change rank at least 3 requires weights change 17.58%
Smallest change to get at least 2 projects to change rank at least 4 requires weights change 19.13%
Smallest change to get at least 2 projects to change rank at least 5 requires weights change 38.51%




## Some needed styling to make the visualization below work

In [25]:
%%HTML
<style>
.flex-container {
    display: flex;
    flex-flox: row wrap;
}
</style>

## Visualization of the results

In [26]:
display(qgrid.show_grid(rval_df))

row_choice = widgets.Dropdown(
    options = df_index,
    description = "Row"
)
col_choice = widgets.Dropdown(
    options = df_columns,
    description = "Column"
)
@interact(col=col_choice, row=row_choice)
def show_it(col,row):
    rank_report_df = pd.DataFrame({
        "Original Ranks":orig_ranks, 
        "New Ranks":new_ranks_df[col][row],
        "Diff": orig_ranks - new_ranks_df[col][row],
    }, index=df.index)
    div1="<div><H2>Weights Information</H2>"+pd.DataFrame({"Original Weights":orig_weights, "New Weights": weights_df[col][row]}).to_html()+"</div>"
    div2="<div style=\"padding-left:40px;\"><h2>Rank Information</h2>"+rank_report_df.to_html()+"</div>"
    total_div='<div class="flex-container">'+div1+div2+"</div>"
    display(HTML(total_div))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

interactive(children=(Dropdown(description='Column', options=('1 Alts', '2 Alts', '3 Alts', '4 Alts', '5 Alts'…